# Safe Support Vector Machine Notebook 
## A Quick Start Guide to implementing Safer Support Vector Machines
### First set some path variables: this notebook expects to find the repository root in your home directory. You can change this by editing the last sys.path.append line to reflect with the repository root is found on your system


In [1]:
import os
import sys
import pylab as plt
import numpy as np
import logging
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn import datasets
from os.path import expanduser

ROOT_DIR = os.path.dirname(os.path.dirname(os.path.abspath("")))
sys.path.append(ROOT_DIR)
home = expanduser("~")
sys.path.append(os.path.abspath(home + "/AI-SDC"))
# sys.path.append(os.path.abspath(".."))

logging.basicConfig()
logger = logging.getLogger("wrapper_svm")
logger.setLevel(logging.INFO)
# ROOT_PROJECT_FOLDER = os.path.dirname(os.path.dirname(__file__))
# sys.path.append(ROOT_PROJECT_FOLDER)
from aisdc.safemodel.classifiers import SafeSVC

2023-06-21 15:27:29.739010: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-21 15:27:29.818950: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-21 15:27:29.822063: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/pulse
2023-06-21 15:27:29.822072: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore a

## Use the sklearn Wisconsin breast cancer dataset

In [2]:
cancer = datasets.load_breast_cancer()
x = np.asarray(cancer.data, dtype=np.float64)
y = np.asarray(cancer.target, dtype=np.float64)

## Kernel for approximator: equivalent to rbf.

In [3]:
def rbf(x, y, gamma=1):
    return np.exp(-gamma * np.sum((x - y) ** 2))


def rbf_svm(x, y, gamma=1):
    r = np.zeros((x.shape[0], y.shape[0]))
    for i in range(x.shape[0]):
        for j in range(y.shape[0]):
            r[i, j] = rbf(x[i, :], y[j, :], gamma)
    return r

## Set parameters

In [4]:
gamma = 0.1  # Kernel width
C = 1  # Penalty term
dhat = 5  # Dimension of approximator
eps = 500  # DP level (not very private)

# Define Differentially Private version with DP level (approximate)

In [5]:
clf3 = SafeSVC(eps=eps, dhat=dhat, C=C, gamma=gamma)
clf3.fit(x, y)
c3 = clf3.predict(x)
p3 = clf3.predict_proba(x)

## Define the model and fit it.
## Save and Request Release
### We are warned that dhat is too low.

In [6]:
clf3 = SafeSVC(eps=eps, dhat=dhat, C=C, gamma=gamma)
clf3.fit(x, y)
clf3.save(name="testSaveSVC.pkl")
clf3.request_release(path="testSaveSVC", ext="pkl")

In [7]:
target_json = os.path.normpath("testSaveSVC/target.json")
with open(target_json, "r") as f:
    print(f.read())

{
    "data_name": "",
    "n_samples": 0,
    "features": {},
    "n_features": 0,
    "n_samples_orig": 0,
    "safemodel": [
        {
            "researcher": "unknown",
            "model_type": "SVC",
            "details": "WARNING: model parameters may present a disclosure risk:\n- parameter dhat = 5 identified as less than the recommended min value of 1000.",
            "recommendation": "Do not allow release",
            "reason": "WARNING: model parameters may present a disclosure risk:\n- parameter dhat = 5 identified as less than the recommended min value of 1000.",
            "timestamp": "2023-06-21 15:27:30"
        }
    ],
    "model_path": "model.pkl",
    "model_name": "SafeSVC",
    "model_params": {}
}


## Set Parameters to safe values

In [8]:
gamma = 0.1  # Kernel width
C = 1  # Penalty term
dhat = 1000  # Dimension of approximator
eps = 500  # DP level (not very private)

## Define the model and fit it.
## Save and Request Release
### Model parameters are within recommended ranges. The saved model can pass through next step of checking procedure

In [9]:
clf3 = SafeSVC(eps=eps, dhat=dhat, C=C, gamma=gamma)
clf3.fit(x, y)
clf3.save(name="testSaveSVC.pkl")
clf3.request_release(path="testSaveSVC", ext="pkl")

## Examine the checkfile

In [10]:
target_json = os.path.normpath("testSaveSVC/target.json")
with open(target_json, "r") as f:
    print(f.read())

{
    "data_name": "",
    "n_samples": 0,
    "features": {},
    "n_features": 0,
    "n_samples_orig": 0,
    "safemodel": [
        {
            "researcher": "unknown",
            "model_type": "SVC",
            "details": "Model parameters are within recommended ranges.\n",
            "recommendation": "Proceed to next step of checking",
            "timestamp": "2023-06-21 15:27:31"
        }
    ],
    "model_path": "model.pkl",
    "model_name": "SafeSVC",
    "model_params": {}
}
